In [86]:
# If don't have, install only once
# !pip install kagglehub

import kagglehub
import pandas as pd
import os

path = kagglehub.dataset_download("snehangsude/audible-dataset")

df = pd.read_csv(os.path.join(path, 'audible_uncleaned.csv'))

# Check first views:
# df.head()
# df.describe()
# df.info()

In [87]:
# Clean text data in Author and Narrator columns
# Remove Writtenby: from the author column
df['author'] = df['author'].str.replace('Writtenby:', '')
# Remove Narratedby: from the narrator column
df['narrator'] = df['narrator'].str.replace('Narratedby:', '')
# df.head(2)

In [88]:
# Clean stars table
# Create separate column for ratings
df['ratings'] = df['stars'].str.extract(r'stars(\d+)\s')
df['ratings'] = df['ratings'].fillna(0).astype('int')
# Create separate column for stars
df['stars out of five'] = df['stars'].str.extract(r'(\d+)\sout')
df['stars out of five'] = df['stars out of five'].fillna(0).astype('float')
df.drop('stars', axis=1, inplace=True)
# df.head(2)
# df['stars out of five'].unique()
# df['ratings'].sample(n=30)

In [89]:
# Replace the comma with ''
df['price'] = df['price'].str.replace(',', '')
# Replace 'Free' with 0
df['price'] = df['price'].str.replace('Free', '0')
# Turn price to float
df['price'] = df['price'].astype(float)
# df['price'].sample(n=30)

In [ ]:
# Convert releasedate to datetime
df['releasedate'] = pd.to_datetime(df['releasedate'])
# Inspect the dataframe 
# df.info()
# Search the entries in the time column for different spellings of min. Let' try min, mins, minutes
# df['time'].str.contains(r'\b(min|mins|minutes)\b', case=False, na=False)

# Replace hrs, mins, and 'Less than 1 minute'
df['time'] = df['time'].str.replace(r'\b(mins?|minutes?)\b', 'min', case=False, regex=True)
df['time'] = df['time'].str.replace(r'\b(hr?|hrs?)\b', 'hr', case=False, regex=True)
df['time'] = df['time'].str.replace('Less than 1 minute', '1 min')
# set(type(x) for x in df['time'])
# df['time'].isna().any()

# Extract the number of hours, turn to integer
df['hours'] = df['time'].str.extract(r'(\d+)\s*hr')
df['hours'] = df['hours'].fillna("0").astype('str')
# Extract the number of minutes, turn to integer
df['mins'] = df['time'].str.extract(r'and\s(\d+)\s*min')
df['mins'] = df['mins'].fillna("0").astype('int')
# Combine hours and minutes into the time_mins column
df['duration'] = df['hours'].astype('str') + ':' + df['mins'].astype('str')
df['mins'] = df['mins'].apply(lambda x: f"0:{x:02}")
# Check the results
df['mins'].unique()

array(['0:20', '0:08', '0:03', '0:16', '0:00', '0:35', '0:23', '0:32',
       '0:56', '0:22', '0:48', '0:01', '0:58', '0:55', '0:25', '0:04',
       '0:41', '0:18', '0:42', '0:12', '0:29', '0:46', '0:37', '0:09',
       '0:44', '0:53', '0:28', '0:26', '0:13', '0:43', '0:39', '0:31',
       '0:19', '0:33', '0:51', '0:47', '0:24', '0:02', '0:10', '0:49',
       '0:27', '0:34', '0:54', '0:06', '0:30', '0:38', '0:36', '0:50',
       '0:11', '0:52', '0:05', '0:21', '0:57', '0:15', '0:45', '0:17',
       '0:59', '0:40', '0:14', '0:07'], dtype=object)